In [1]:
import os
import tvm
from tvm.script import ir as I
from tvm.script import tir as T
from tvm import autotvm, auto_scheduler
from tvm.autotvm.tuner import XGBTuner, GATuner, RandomTuner, GridSearchTuner
from tvm import meta_schedule as ms
from tvm.ir import IRModule
from tvm import relax
from tvm import rpc
from tvm.contrib import utils, ndk
vocab_size = 50000
@I.ir_module
class ModuleToManual:
    @T.prim_func
    def fused_decode5_fused_matmul6_multiply1(
        lv1617: T.Buffer((T.int64(512), T.int64(11008)), "uint32"),
        lv1618: T.Buffer((T.int64(128), T.int64(11008)), "float16"),
        lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
        lv4: T.Buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16"),
        p_output0_intermediate: T.Buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        ),
    ):
        T.func_attr({"tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(11008)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        )
        for i, j in T.grid(T.int64(4096), T.int64(11008)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1617[v_i // T.int64(8), v_j], lv1618[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1617[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1618[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(11008), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("T_multiply"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(
                    lv4[v_ax0, v_ax1, v_ax2], var_matmul_intermediate[v_ax0, v_ax1, v_ax2]
                )
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = (
                    lv4[v_ax0, v_ax1, v_ax2] * var_matmul_intermediate[v_ax0, v_ax1, v_ax2]
                )

    @T.prim_func
    def fused_decode5_fused_matmul6_silu1(
        lv1611: T.Buffer((T.int64(512), T.int64(11008)), "uint32"),
        lv1612: T.Buffer((T.int64(128), T.int64(11008)), "float16"),
        lv1622: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"),
        p_output0_intermediate: T.Buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        ),
    ):
        T.func_attr({"tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_decode_intermediate = T.alloc_buffer((T.int64(4096), T.int64(11008)), "float16")
        var_matmul_intermediate = T.alloc_buffer(
            (T.int64(1), T.int64(1), T.int64(11008)), "float16"
        )
        compute = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(11008)), "float16")
        for i, j in T.grid(T.int64(4096), T.int64(11008)):
            with T.block("decode"):
                v_i, v_j = T.axis.remap("SS", [i, j])
                T.reads(lv1611[v_i // T.int64(8), v_j], lv1612[v_i // T.int64(32), v_j])
                T.writes(var_decode_intermediate[v_i, v_j])
                var_decode_intermediate[v_i, v_j] = (
                    T.Cast(
                        "float16",
                        T.bitwise_and(
                            T.shift_right(
                                lv1611[v_i // T.int64(8), v_j],
                                T.Cast("uint32", v_i % T.int64(8)) * T.uint32(4),
                            ),
                            T.uint32(15),
                        ),
                    )
                    - T.float16(7)
                ) * lv1612[v_i // T.int64(32), v_j]
        for i0, i1, i2, k in T.grid(T.int64(1), T.int64(1), T.int64(11008), T.int64(4096)):
            with T.block("matmul"):
                v_i0, v_i1, v_i2, v_k = T.axis.remap("SSSR", [i0, i1, i2, k])
                T.reads(lv1622[v_i0, v_i1, v_k], var_decode_intermediate[v_k, v_i2])
                T.writes(var_matmul_intermediate[v_i0, v_i1, v_i2])
                with T.init():
                    var_matmul_intermediate[v_i0, v_i1, v_i2] = T.float16(0)
                var_matmul_intermediate[v_i0, v_i1, v_i2] = (
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                    + lv1622[v_i0, v_i1, v_k] * var_decode_intermediate[v_k, v_i2]
                )
        for i0, i1, i2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("compute"):
                v_i0, v_i1, v_i2 = T.axis.remap("SSS", [i0, i1, i2])
                T.reads(var_matmul_intermediate[v_i0, v_i1, v_i2])
                T.writes(compute[v_i0, v_i1, v_i2])
                compute[v_i0, v_i1, v_i2] = T.sigmoid(
                    var_matmul_intermediate[v_i0, v_i1, v_i2]
                )
        for ax0, ax1, ax2 in T.grid(T.int64(1), T.int64(1), T.int64(11008)):
            with T.block("T_multiply"):
                v_ax0, v_ax1, v_ax2 = T.axis.remap("SSS", [ax0, ax1, ax2])
                T.reads(
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2],
                    compute[v_ax0, v_ax1, v_ax2],
                )
                T.writes(p_output0_intermediate[v_ax0, v_ax1, v_ax2])
                p_output0_intermediate[v_ax0, v_ax1, v_ax2] = (
                    var_matmul_intermediate[v_ax0, v_ax1, v_ax2]
                    * compute[v_ax0, v_ax1, v_ax2]
                )



In [82]:
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def print_object_attr(cls):
    print(f"attr len = {len(cls.__dict__)}")
    for k, v in cls.__dict__.items():
        # if k.startswith("_") or k.startswith("__"):
        #     continue
        print(f"{k}: {v}")

def sch_fused_decode5_fused_matmul6_multiply1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    v7, v8, v9 = sch.sample_perfect_tile(
        loop=l6, n=3, max_innermost_factor=4, decision=[43, 64, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[128, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)
    b20 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    b21 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    b22 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b22, loop=l11, preserve_unit_loops=True, index=-1)
    v23 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch", ann_val=v23
    )
    sch.compute_at(block=b20, loop=l17, preserve_unit_loops=True, index=-1)
    sch.compute_at(block=b21, loop=l16, preserve_unit_loops=True, index=-1)
    l24, l25, l26, l27, l28, l29 = sch.get_loops(block=b20)
    sch.vectorize(loop=l29)
    l30, l31, l32, l33, l34 = sch.get_loops(block=b21)
    sch.vectorize(loop=l34)
    l35, l36, l37, l38, l39 = sch.get_loops(block=b19)
    sch.vectorize(loop=l39)
    sch.vectorize(loop=l12)
    b40 = sch.decompose_reduction(block=b1, loop=l16)
    b41 = sch.get_block(name="T_multiply", func_name="main")
    sch.reverse_compute_inline(block=b41)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch")
    l42, l43, l44, l45, l46 = sch.get_loops(block=b22)
    l47, l48, l49 = sch.split(loop=l46, factors=[None, 64, 8], preserve_unit_iters=True)
    sch.vectorize(loop=l49)
    sch.bind(loop=l48, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

def sch_fused_decode5_fused_matmul6_silu1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    v7, v8, v9 = sch.sample_perfect_tile(
        loop=l6, n=3, max_innermost_factor=4, decision=[43, 64, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[128, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)
    b20 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    b21 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    b22 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b22, loop=l11, preserve_unit_loops=True, index=-1)
    v23 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch", ann_val=v23
    )
    sch.compute_at(block=b20, loop=l17, preserve_unit_loops=True, index=-1)
    sch.compute_at(block=b21, loop=l16, preserve_unit_loops=True, index=-1)
    l24, l25, l26, l27, l28, l29 = sch.get_loops(block=b20)
    sch.vectorize(loop=l29)
    l30, l31, l32, l33, l34 = sch.get_loops(block=b21)
    sch.vectorize(loop=l34)
    l35, l36, l37, l38, l39 = sch.get_loops(block=b19)
    sch.vectorize(loop=l39)
    sch.vectorize(loop=l12)
    b40 = sch.decompose_reduction(block=b1, loop=l16)
    b41 = sch.get_block(name="compute", func_name="main")
    sch.compute_inline(block=b41)
    b42 = sch.get_block(name="T_multiply", func_name="main")
    sch.reverse_compute_inline(block=b42)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b22, ann_key="meta_schedule.cooperative_fetch")
    l43, l44, l45, l46, l47 = sch.get_loops(block=b22)
    l48, l49, l50 = sch.split(loop=l47, factors=[None, 64, 8], preserve_unit_iters=True)
    sch.vectorize(loop=l50)
    sch.bind(loop=l49, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

sch_manual = tvm.tir.Schedule(ModuleToManual)

sch_manual.mod['main'] = sch_fused_decode5_fused_matmul6_silu1(sch_manual.mod['main'])
print(sch_manual.mod.script())

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(lv1323: T.Buffer((T.int64(512), T.int64(50000)), "uint32"), lv1324: T.Buffer((T.int64(128), T.int64(50000)), "float16"), lv1607: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(50000)), "float32")):
        T.func_attr({"global_symbol": "main", "tir.is_scheduled": 1, "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_matmul_intermediate_local = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(50000)), "float16", scope="local")
        lv1323_local = T.alloc_buffer((T.int64(512), T.int64(50000)), "uint32", scope="local")
        lv1324_local = T.alloc_buffer((T.int64(128), T.int64(50000)), "float16", scope="local")
        lv1607_shared = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16", scope="shared")
        for i0_i1_i2_fused_0 in T.thread_bin

In [3]:
def merge_from_cuda(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    b2 = sch.get_block(name="compute", func_name="main")
    b3 = sch.get_block(name="root", func_name="main")
    sch.annotate(block_or_loop=b1, ann_key="meta_schedule.tiling_structure", ann_val="SSSRRSRS")
    l4, l5, l6, l7 = sch.get_loops(block=b1)
    v8, v9, v10, v11, v12 = sch.sample_perfect_tile(loop=l4, n=5, max_innermost_factor=64, decision=[1, 1, 1, 1, 1])
    l13, l14, l15, l16, l17 = sch.split(loop=l4, factors=[v8, v9, v10, v11, v12], preserve_unit_iters=True)
    v18, v19, v20, v21, v22 = sch.sample_perfect_tile(loop=l5, n=5, max_innermost_factor=64, decision=[1, 1, 1, 1, 1])
    l23, l24, l25, l26, l27 = sch.split(loop=l5, factors=[v18, v19, v20, v21, v22], preserve_unit_iters=True)
    v28, v29, v30, v31, v32 = sch.sample_perfect_tile(loop=l6, n=5, max_innermost_factor=64, decision=[781, 2, 32, 1, 1])
    l33, l34, l35, l36, l37 = sch.split(loop=l6, factors=[v28, v29, v30, v31, v32], preserve_unit_iters=True)
    v38, v39, v40 = sch.sample_perfect_tile(loop=l7, n=3, max_innermost_factor=64, decision=[128, 2, 16])
    l41, l42, l43 = sch.split(loop=l7, factors=[v38, v39, v40], preserve_unit_iters=True)
    sch.reorder(l13, l23, l33, l14, l24, l34, l15, l25, l35, l41, l42, l16, l26, l36, l43, l17, l27, l37)
    l44 = sch.fuse(l13, l23, l33, preserve_unit_iters=True)
    sch.bind(loop=l44, thread_axis="blockIdx.x")
    l45 = sch.fuse(l14, l24, l34, preserve_unit_iters=True)
    sch.bind(loop=l45, thread_axis="vthread.x")
    l46 = sch.fuse(l15, l25, l35, preserve_unit_iters=True)
    sch.bind(loop=l46, thread_axis="threadIdx.x")
    sch.annotate(block_or_loop=b1, ann_key="meta_schedule.thread_extent_low_inclusive", ann_val=32)
    sch.annotate(block_or_loop=b1, ann_key="meta_schedule.thread_extent_high_inclusive", ann_val=1024)
    b47 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b47, loop=l46, preserve_unit_loops=True, index=-1)
    b48 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared", consumer_blocks=[b1])
    sch.compute_at(block=b48, loop=l41, preserve_unit_loops=True, index=-1)
    l49, l50, l51, l52, l53, l54, l55 = sch.get_loops(block=b48)
    l56 = sch.fuse(l53, l54, l55, preserve_unit_iters=True)
    v57 = sch.sample_categorical(candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=1)
    sch.annotate(block_or_loop=b48, ann_key="meta_schedule.cooperative_fetch", ann_val=v57)
    b58 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="shared", consumer_blocks=[b1])
    sch.compute_at(block=b58, loop=l41, preserve_unit_loops=True, index=-1)
    l59, l60, l61, l62, l63, l64 = sch.get_loops(block=b58)
    l65 = sch.fuse(l63, l64, preserve_unit_iters=True)
    v66 = sch.sample_categorical(candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=1)
    sch.annotate(block_or_loop=b58, ann_key="meta_schedule.cooperative_fetch", ann_val=v66)
    sch.reverse_compute_inline(block=b2)
    sch.compute_inline(block=b0)
    v67 = sch.sample_categorical(candidates=[0, 16, 64, 512, 1024], probs=[0.20000000000000001, 0.20000000000000001, 0.20000000000000001, 0.20000000000000001, 0.20000000000000001], decision=4)
    sch.annotate(block_or_loop=b3, ann_key="meta_schedule.unroll_explicit", ann_val=v67)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b48, ann_key="meta_schedule.cooperative_fetch")
    l68, l69, l70, l71, l72 = sch.get_loops(block=b48)
    l73, l74, l75 = sch.split(loop=l72, factors=[None, 32, 2], preserve_unit_iters=True)
    sch.vectorize(loop=l75)
    sch.bind(loop=l74, thread_axis="threadIdx.x")
    sch.unannotate(block_or_loop=b58, ann_key="meta_schedule.cooperative_fetch")
    l76, l77, l78, l79, l80 = sch.get_loops(block=b58)
    l81, l82, l83 = sch.split(loop=l80, factors=[None, 32, 2], preserve_unit_iters=True)
    sch.vectorize(loop=l83)
    sch.bind(loop=l82, thread_axis="threadIdx.x")
    b84 = sch.get_block(name="root", func_name="main")
    sch.unannotate(block_or_loop=b84, ann_key="meta_schedule.unroll_explicit")
    b85, b86, b87, b88 = sch.get_child_blocks(b84)
    l89, l90, l91, l92, l93, l94, l95 = sch.get_loops(block=b85)
    l96, l97, l98, l99, l100, l101, l102 = sch.get_loops(block=b86)
    l103, l104, l105, l106, l107, l108, l109, l110, l111, l112, l113, l114 = sch.get_loops(block=b87)
    sch.annotate(block_or_loop=l103, ann_key="pragma_auto_unroll_max_step", ann_val=1024)
    sch.annotate(block_or_loop=l103, ann_key="pragma_unroll_explicit", ann_val=1)
    l115, l116, l117, l118, l119, l120 = sch.get_loops(block=b88)
    b121 = sch.get_block(name="matmul", func_name="main")
    l122, l123, l124, l125, l126, l127, l128, l129, l130, l131, l132, l133 = sch.get_loops(block=b121)
    b134 = sch.decompose_reduction(block=b121, loop=l125)
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

    
sch_manual = tvm.tir.Schedule(ModuleToManual)

sch_manual.mod['main'] = merge_from_cuda(sch_manual.mod['main'])
print(sch_manual.mod.script())

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(lv1323: T.Buffer((T.int64(512), T.int64(49984)), "uint32"), lv1324: T.Buffer((T.int64(128), T.int64(49984)), "float16"), lv1607: T.Buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16"), p_output0_intermediate: T.Buffer((T.int64(1), T.int64(1), T.int64(49984)), "float32")):
        T.func_attr({"global_symbol": "main", "tir.is_scheduled": 1, "tir.noalias": T.bool(True)})
        # with T.block("root"):
        var_matmul_intermediate_local = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(49984)), "float16", scope="local")
        lv1607_shared = T.alloc_buffer((T.int64(1), T.int64(1), T.int64(4096)), "float16", scope="shared")
        p_output0_intermediate_1_shared = T.alloc_buffer((T.int64(4096), T.int64(49984)), "float16", scope="shared")
        for i0_0_i1_0_i2_0_fused in T.thread_binding(T.int64(781), thread="blockIdx.x", annotations={"pragma_auto_unro

In [83]:
# run and compare with cuda
import numpy as np
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
Output_np = np.empty((1, 1, vocab_size)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)


cuda -keys=cuda,gpu -arch=sm_61 -max_num_threads=1024 -max_shared_memory_per_block=49152 -max_threads_per_block=1024 -registers_per_block=65536 -thread_warp_size=32


/tmp/ipykernel_15219/584062030.py:28: RuntimeWarning: overflow encountered in cast
  Output_np = np.empty((1, 1, vocab_size)).astype("float16")


In [ ]:

# cuda未优化版本测试
manual_rt_mod = tvm.build(sch_manual.mod, target="cuda")
manual_evaluator = manual_rt_mod.time_evaluator("main", dev, number=100)
print("manual_evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / manual_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))

In [ ]:
# cuda优化版本测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
rt_mod = tvm.build(src_gpu_mod, target="cuda")
# print(rt_mod.get_source())
# evaluator = rt_mod.time_evaluator("main", dev, number=100)
# print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))

In [84]:
os.environ["TVM_NDK_CC"]="/home/sensetime/Android/Sdk/ndk/25.2.9519653/toolchains/llvm/prebuilt/linux-x86_64/bin/aarch64-linux-android33-clang++"
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.4.236.32"
rpc_port = 9190
comp_target = tvm.target.Target("opencl", host="llvm -mtriple=aarch64-linux-android")  # TODO: Only support arm64 for now

def test_opencl(mod: tvm.IRModule, name_hint: str):
    # mod = tvm.lower(sch_manual.mod)
    print("Build ...")
    android_rt_mod = tvm.build(mod, target="opencl", target_host="llvm -mtriple=aarch64-linux-android")
    print(android_rt_mod.imported_modules[0].get_source())
    temp = utils.tempdir()
    path_dso_cl = temp.relpath("dev_lib_cl.so")
    android_rt_mod.export_library(path_dso_cl, ndk.create_shared)

    print("Run GPU(OpenCL Flavor) test ...")
    # Establish remote connection with target hardware

    tracker = rpc.connect_tracker(rpc_host, rpc_port)
    remote = tracker.request(device_key, priority=0, session_timeout=60)
    print("Connect to device done.")
    dev = remote.cl(0)
    remote.upload(path_dso_cl)
    f1 = remote.load_module("dev_lib_cl.so")

    W_nd = tvm.nd.array(W_np, dev)
    S_nd = tvm.nd.array(S_np, dev)
    Input_nd = tvm.nd.array(Input_np, dev)
    Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
    test_number=10
    time_f = f1.time_evaluator(f1.entry_name, dev, number=test_number)
    cost = time_f(W_nd, S_nd, Input_nd, Output_nd).mean
    print("evaluator[%s] GEMV-Blocking: %f ms with loop %d" % (name_hint, cost * 1000, test_number))
    print("evaluator[%s] GEMV-Blocking: %f GFLOPS" % (name_hint, num_flop / cost / 1e9))

    return Output_nd.numpy()

In [85]:
# 未优化版本opencl测试
sch = tvm.tir.Schedule(ModuleSrc)
with target:
    src_gpu_mod = tvm.tir.transform.DefaultGPUSchedule()(sch.mod) ##
src_output = test_opencl(src_gpu_mod, "source")
print(src_output)


Build ...
// Function: main_kernel_2
#ifdef cl_khr_fp16
#pragma OPENCL EXTENSION cl_khr_fp16 : enable
#elif defined(cl_amd_fp16)
#pragma OPENCL EXTENSION cl_amd_fp16 : enable
#else
#error "Half precision floating point not supported by OpenCL implementation on your device." 
#endif

__kernel void main_kernel_2(__global float* restrict p_output0_intermediate, __global half* restrict var_matmul_intermediate) {
  if ((((convert_int(get_group_id(0))) * 16) + ((convert_int(get_local_id(0))) >> 4)) < 3125) {
    p_output0_intermediate[(((convert_int(get_group_id(0))) * 256) + (convert_int(get_local_id(0))))] = (convert_float(var_matmul_intermediate[(((convert_int(get_group_id(0))) * 256) + (convert_int(get_local_id(0))))]));
  }
}

// Function: main_kernel_1
#ifdef cl_khr_fp16
#pragma OPENCL EXTENSION cl_khr_fp16 : enable
#elif defined(cl_amd_fp16)
#pragma OPENCL EXTENSION cl_amd_fp16 : enable
#else
#error "Half precision floating point not supported by OpenCL implementation on your device."

/data/workspace/llm/github/new_wksp/tvm-unity/python/tvm/driver/build_module.py:264: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


Connect to device done.
evaluator[source] GEMV-Blocking: 109.702528 ms with loop 10
evaluator[source] GEMV-Blocking: 11.197616 GFLOPS
[[[-6856. -6048. -5840. ... -6392. -6748. -6924.]]]


In [88]:
#优化版本opencl测试
opt_output = test_opencl(sch_manual.mod, "opted")
print(opt_output)
np.testing.assert_equal(opt_output, src_output)

Build ...
// Function: main_kernel
#ifdef cl_khr_fp16
#pragma OPENCL EXTENSION cl_khr_fp16 : enable
#elif defined(cl_amd_fp16)
#pragma OPENCL EXTENSION cl_amd_fp16 : enable
#else
#error "Half precision floating point not supported by OpenCL implementation on your device." 
#endif

__kernel void main_kernel(__global uint* restrict lv1323, __global half* restrict lv1324, __global half* restrict lv1607, __global float* restrict p_output0_intermediate) {
  __local half lv1607_shared[4096];
  half4 var_matmul_intermediate_local[1];
  half4 lv1324_local[1];
  uint4 lv1323_local[1];
  for (int ax2_0 = 0; ax2_0 < 41; ++ax2_0) {
    for (int ax2_2_s = 0; ax2_2_s < 2; ++ax2_2_s) {
      if (((ax2_0 * 25) + ((convert_int(get_local_id(0))) >> 1)) < 1024) {
        lv1607_shared[(((ax2_0 * 100) + ((convert_int(get_local_id(0))) * 2)) + ax2_2_s)] = lv1607[(((ax2_0 * 100) + ((convert_int(get_local_id(0))) * 2)) + ax2_2_s)];
      }
    }
  }
  var_matmul_intermediate_local[0] = ((half4)((half)0.00000

KeyboardInterrupt: 

In [ ]:
import numpy as np
target = tvm.target.Target("opencl -device=adreno", host="llvm -mtriple=aarch64-linux-gnu")
device_key="android"
rpc_host = "10.158.176.30"
rpc_port = 5001
# remote = autotvm.measure.request_remote(device_key, "10.158.176.30", 5001, timeout=10000)
# dev = remote.device(str(target), 0)

# num_flop = 1228406784
# W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
# S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
# Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# # Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# W_nd = tvm.nd.array(W_np, dev)
# S_nd = tvm.nd.array(S_np, dev)
# Input_nd = tvm.nd.array(Input_np, dev)
# Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)

In [ ]:
rpc_config = ms.runner.RPCConfig(tracker_host=rpc_host, tracker_port=rpc_port, tracker_key = device_key)
runner= ms.runner.RPCRunner(rpc_config)
# ms.builder.LocalBuilder()
sch = tvm.tir.Schedule(ModuleSrc)
database = ms.tune_tir(
    mod=ModuleSrc,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_first",
    cost_model="xgb",
    runner = runner
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, sch.mod, target)
print(type(sch1))

In [ ]:
from tvm.script import relax as R
@I.ir_module
class Module:
    @R.function
    def main(A: R.Tensor((3, 4), dtype="float32"), B: R.Tensor((4, 5), dtype="float32")):
        with R.dataflow():
            lv: R.Tensor((3, 5), dtype="float32") = R.matmul(A, B)
            gv: R.Tensor((3, 5), dtype="float32") = lv
            R.output(gv)
        return gv

In [ ]:
## auto_scheduler test
from tvm import auto_scheduler
import numpy as np
a_np = np.random.rand(3, 4).astype("float32")
b_np = np.random.rand(4, 5).astype("float32")
a_nd = tvm.runtime.NDArray(a_np)
b_nd = tvm.runtime.NDArray(b_np)
sch = tvm.tir.Schedule(Module)

params = {"A": a_np, "B": b_np}
## 报错，这里只支持relay
# tasks = auto_scheduler.extract_tasks(sch.mod, params, target=target)
tasks = ms.relax_integration.extract_tasks(sch.mod, target=target, params=params)
print(len(tasks))

In [ ]:

from mod_deploy import Module as ModuleAll
params_all = {}
tasks_all = auto_scheduler.extract_tasks(ModuleAll, params_all, target=target)
print(len(tasks_all))

In [ ]:
import numpy as np
log_file = "tune.json"
def _detect_local_cuda():
    dev = tvm.cuda()
    if not dev.exist:
        return None
    return tvm.target.Target(
        {
            "kind": "cuda",
            "max_shared_memory_per_block": dev.max_shared_memory_per_block,
            "max_threads_per_block": dev.max_threads_per_block,
            "thread_warp_size": dev.warp_size,
            "registers_per_block": 65536,
            "arch": "sm_" + tvm.cuda().compute_version.replace(".", ""),
        }
    )
# target = tvm.target.Target("cuda", host="llvm")
target = _detect_local_cuda()

print(target)
# 定义计算任务
dev = tvm.cuda(0)

num_flop = 1228406784
W_np = np.random.uniform(size=(512, vocab_size)).astype("uint32")
S_np = np.random.uniform(size=(128, vocab_size)).astype("float16")
Input_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
# Output_np = np.random.uniform(size=(1, 1, 4096)).astype("float16")
W_nd = tvm.nd.array(W_np, dev)
S_nd = tvm.nd.array(S_np, dev)
Input_nd = tvm.nd.array(Input_np, dev)
Output_nd = tvm.nd.array(np.zeros((1, 1, vocab_size), dtype="float32"), dev)
sch = tvm.tir.Schedule(ModuleSrc)
new_mod = sch.mod


In [ ]:
# task = auto_scheduler.SearchTask(func=sch.mod['fused_fused_decode11_fused_matmul5_cast2'], args=sch.mod['fused_fused_decode11_fused_matmul5_cast2'].params, target=target)

# tune_option = auto_scheduler.TuningOptions(
#     num_measure_trials=10,
#     measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
#     verbose=2,
# )


database = ms.tune_tir(
    mod=new_mod,
    target=target,
    max_trials_global=64,
    num_trials_per_iter=64,
    work_dir="./tune_45593_1",
    cost_model="xgb"
)
print(len(database))
sch1 = ms.tir_integration.compile_tir(database, new_mod, target)
print(type(sch1))

In [ ]:
# print(sch1.trace)
# print(sch1.mod.script())
rt_mod = tvm.build(sch1.mod, target="cuda")

evaluator = rt_mod.time_evaluator("main", dev, number=100)

print("evaluator GEMV-Blocking: %f GFLOPS" % (1228406784 / evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))




In [ ]:

record_database = ms.Database.create(kind='json', work_dir='./tune_45593_1')


In [ ]:
record_sch = ms.tir_integration.compile_tir(record_database, new_mod, target)

record_rt_mod = tvm.build(record_sch.mod, target="cuda")

record_evaluator = record_rt_mod.time_evaluator("main", dev, number=20)

print("evaluator GEMV-Blocking: %f GFLOPS" % (num_flop / record_evaluator(W_nd, S_nd, Input_nd, Output_nd).mean / 1e9))
print(record_sch.trace)
print(record_sch.mod.script())

In [ ]:
from typing import TYPE_CHECKING, Dict, List, Optional, Union, Callable
from tvm import runtime
if TYPE_CHECKING:
    import numpy as np  # type: ignore
    from tvm.ir import IRModule
    from tvm.meta_schedule.runner import EvaluatorConfig, RPCConfig
    from tvm.runtime import Device, Module, NDArray
    from tvm.target import Target
    from tvm.runtime.vm import Executable


def f_measurement(
    rt_mod: runtime.Module, device: runtime.ndarray.Device, input_data: Dict[str, runtime.NDArray]
):
    vm = relax.VirtualMachine(rt_mod, device=device)
    vm.save_function("main", "measure_func", **input_data, include_return=False)
    evaluator = vm.time_evaluator(
        func_name="measure_func",
        dev=device,
        repeat=100,
        number=1,
        min_repeat_ms=500,
    )
    return evaluator()

def run_module_via_rpc(
    rpc_config: "RPCConfig",
    lib: Union["Module", "Executable"],
    dev_type: str,
    args: Union[Dict[int, "np.ndarray"], Dict[str, "np.ndarray"]],
    continuation: Callable,
    backend: Optional[str] = "graph",
):
    """Execute a tvm.runtime.Module on RPC remote"""
    # pylint: disable=import-outside-toplevel
    import os
    import tempfile

    from tvm.contrib.tar import tar
    from tvm.runtime import ndarray

    # pylint: enable=import-outside-toplevel

    with tempfile.TemporaryDirectory() as tmp_dir:
        # filename = os.path.join(tmp_dir, "tvm_tmp_mod." + tar.output_format)
        filename = os.path.join(tmp_dir, "tvm_tmp_mod." + "so")
        if backend == "vm":
            code, lib = lib.save(filename, fmt="so")
        from tvm.contrib import ndk
        lib.export_library(filename, ndk.create_shared)
        session = rpc_config.connect_server()
        print(type(session._sess))
        session.upload(filename)
        _, filename = os.path.split(filename)
        rt_mod = session.load_module(filename)
        
        if backend == "vm":
            rt_mod = session.get_function("runtime.Load_Executable")(code, rt_mod)
            # rt_mod = session.get_function("runtime.module.loadfile_relax.Executable")(filename)
        dev = session.device(dev_type=dev_type, dev_id=0)
        # print(dev)
        # create the remote runtime module
        print(rt_mod)
        print(rt_mod['main'])
        from tvm.contrib import graph_executor as runtime
        module = runtime.GraphModule(rt_mod["main"](dev))
        print(module)
        for k, v in args.items():
            module.set_input(k, tvm.nd.array(v))
        return module.run()
        # nd_args = {k: ndarray.array(v, dev) for k, v in args.items()}
        nd_args = {k: ndarray.empty(v.shape, v.dtype, dev) for k, v in args.items()}
        return continuation(rt_mod, dev, nd_args)